In [1]:
import unicodedata
import unidecode
import requests
import pandas as pd
import json
import numpy as np
from itertools import chain
from urllib.request import urlopen
from bs4 import BeautifulSoup

#Output Folder Data Path
path = "output_data/"

#Read_csv_Path
nba_players_info_path = f"{path}nba_players_info.csv"
bkr_players_info_letter_path = f"{path}bkr_players_info_ByLetter.csv"
bkr_players_info_origin_path = f"{path}bkr_players_info_ByOrigin.csv"
bkr_players_2021_22_path = f"{path}teams_players_2021_2022.csv"
nba_stats_path = f"{path}nba_stats.csv"



In [2]:
# Read the mouse data and the study results
nba_players_info = pd.read_csv(nba_players_info_path)
nba_players_info = nba_players_info.iloc[: , 1:]
bkr_players_info_letter = pd.read_csv(bkr_players_info_letter_path)
bkr_players_info_letter = bkr_players_info_letter.iloc[: , 1:]
bkr_players_info_origin = pd.read_csv(bkr_players_info_origin_path)
bkr_players_info_origin = bkr_players_info_origin.iloc[: , 1:]
bkr_players_2021_22 = pd.read_csv(bkr_players_2021_22_path)
nba_stats = pd.read_csv(nba_stats_path)


In [3]:
bkr_player_merge1 = bkr_players_info_letter[['Player','From','To','Colleges','HTML Reference']]
bkr_player_merge2 = bkr_players_info_origin[['Player','Date','City','Country','State']]

bkr_player_merge1 = bkr_player_merge1.astype({'Player':'string'})
for i in range(len(bkr_player_merge1)):
     bkr_player_merge1.loc[i,'Player'] = unidecode.unidecode(bkr_player_merge1.loc[i,'Player'])

bkr_player_merge2 = bkr_player_merge2.astype({'Player':'string'})
for i in range(len(bkr_player_merge2)):
     bkr_player_merge2.loc[i,'Player'] = unidecode.unidecode(bkr_player_merge2.loc[i,'Player'])


bkr_players_info_complete = pd.merge(bkr_player_merge1, bkr_player_merge2, how='outer', on = 'Player')

bkr_players_info_complete = bkr_players_info_complete.astype({'Colleges':'string'})
bkr_players_info_complete['Colleges'].str.strip()

bkr_players_no_college = bkr_players_info_complete[bkr_players_info_complete['Colleges'].isna()].reset_index(drop=True)
bkr_players_college = bkr_players_info_complete.loc[bkr_players_info_complete['Colleges'] != ''].reset_index(drop=True)


bkr_players_info_complete.to_csv(f'{path}bkr_players_info_complete.csv', encoding='utf-8-sig')
bkr_players_college.to_csv(f'{path}bkr_players_college.csv', encoding='utf-8-sig')
bkr_players_no_college.to_csv(f'{path}bkr_players_no_college.csv', encoding='utf-8-sig')




In [4]:
bkr_2022_merge1 = bkr_players_info_complete[['Player','Colleges','Date','City','Country','State']]
bkr_2022_merge2 = bkr_players_2021_22[['Player Name','Team Name','City']]

bkr_2022_player_origin_lookup = pd.merge(bkr_2022_merge1, bkr_2022_merge2, how='right', left_on = 'Player', right_on = 'Player Name')

bkr_2022_player_origin_lookup.pop('Player')

first_column = bkr_2022_player_origin_lookup.pop('Player Name') 
bkr_2022_player_origin_lookup.insert(0, 'Player', first_column)

bkr_2022_player_origin_lookup.columns = ['Player', 'Colleges', 'Date', 'Player City', 'Country', 'State', 'Team Name', 'Team City']

bkr_2022_player_origin_lookup.to_csv(f'{path}bkr_2022_player_origin_lookup.csv', encoding='utf-8-sig')


In [5]:
nba_2022_player_origin_lookup = bkr_2022_player_origin_lookup.copy()

nba_2022_player_origin_lookup = nba_2022_player_origin_lookup.astype({'Player':'string'})
nba_2022_player_origin_lookup = nba_2022_player_origin_lookup.astype({'Country':'string'})

nba_2022_player_origin_lookup['Player'].str.strip()
nba_2022_player_origin_lookup['Country'].str.strip()
nba_2022_player_origin_lookup['Colleges'].str.strip()

nba_2022_player_origin_lookup[['Colleges', 'Country']] = nba_2022_player_origin_lookup[['Colleges','Country']].fillna('')

player=''
college=''
country=''

for i, row in nba_2022_player_origin_lookup.iterrows():
    
    try:
        if nba_2022_player_origin_lookup.loc[i, 'Colleges'] == '':
            player = nba_2022_player_origin_lookup['Player'][i]
            college = nba_players_info.loc[nba_players_info['DISPLAY_FIRST_LAST'] == player, 'SCHOOL'].iloc[0]
            
            nba_2022_player_origin_lookup.loc[i, 'Colleges'] = college
            
            print(f'{player} college info updated')
    except Exception as e:
        print(f'{player} college info not found or {e} | Skipping...')
        pass
    
    try:
        if nba_2022_player_origin_lookup.loc[i, 'Country'] == '':
            player = nba_2022_player_origin_lookup['Player'][i]
            country = nba_players_info.loc[nba_players_info['DISPLAY_FIRST_LAST'] == player, 'COUNTRY'].iloc[0]
            
            nba_2022_player_origin_lookup.loc[i, 'Country'] = country
            
            print(f'{player} country info updated')
    except Exception as e:
        print(f'{player} country info not found or {e} | Skipping...')
        pass

nba_2022_player_origin_lookup.to_csv(f'{path}nba_2022_player_origin_lookup.csv', encoding='utf-8-sig')

Kevin Knox II college info updated
Kevin Knox II country info updated
Lou Williams college info updated
Timothe Luwawu-Cabarrot college info updated
Danilo Gallinari college info updated
Bogdan Bogdanovic college info updated
Clint Capela college info updated
Daniel Theis college info not found or single positional indexer is out-of-bounds | Skipping...
Robert Williams III college info updated
Robert Williams III country info updated
Cedi Osman college info updated
Willy Hernangomez college info updated
Jonas Valanciunas college info updated
Marko Simonovic college info updated
Frank Ntilikina college info updated
Maxi Kleber college info not found or single positional indexer is out-of-bounds | Skipping...
Davis Bertans college info updated
Boban Marjanovic college info updated
Luka Doncic college info not found or single positional indexer is out-of-bounds | Skipping...
Facundo Campazzo college info not found or single positional indexer is out-of-bounds | Skipping...
Nikola Jokic co

In [6]:
player_stats_2022_merge1 = nba_2022_player_origin_lookup
player_stats_2022_merge2 = nba_stats[['Player','PER']]

player_stats_2022_merge3 = player_stats_2022_merge2.drop_duplicates(subset=['Player'], keep='first')

player_stats_2022 = pd.merge(player_stats_2022_merge1, player_stats_2022_merge3, how='left', left_on = 'Player', right_on = 'Player')

player_stats_2022.to_csv(f'{path}player_stats_2022.csv', encoding='utf-8-sig')

In [7]:
player_stats_2022_single_college_split = player_stats_2022.copy()
player_stats_2022_single_college_split['Colleges'] = player_stats_2022_single_college_split['Colleges'].str.split().agg("".join)

# return list from series of comma-separated strings
def chainer(s):
    return list(chain.from_iterable(s.str.split(',')))

# calculate lengths of splits
lens = player_stats_2022_single_college_split['Colleges'].str.split(',').map(len)

# create new dataframe, repeating or chaining as appropriate
player_stats_2022_single_college = pd.DataFrame({
                    'Player': np.repeat(player_stats_2022_single_college_split['Player'], lens),
                    'Colleges': chainer(player_stats_2022_single_college_split['Colleges']),
                    'Date': np.repeat(player_stats_2022_single_college_split['Date'], lens),
                    'Player City': np.repeat(player_stats_2022_single_college_split['Player City'], lens),
                    'Country': np.repeat(player_stats_2022_single_college_split['Country'], lens),
                    'State': np.repeat(player_stats_2022_single_college_split['State'], lens),
                    'Team Name': np.repeat(player_stats_2022_single_college_split['Team Name'], lens),                                                                              
                    'Team City': np.repeat(player_stats_2022_single_college_split['Team City'], lens),
                    'PER': np.repeat(player_stats_2022_single_college_split['PER'], lens),                   
                    })


player_stats_2022_college_filter = player_stats_2022_single_college.loc[(player_stats_2022_single_college['Colleges'] != '') & (player_stats_2022_single_college['PER'] > 0)].reset_index(drop=True)

player_stats_2022_college_clean= player_stats_2022_college_filter[['Player','Colleges','PER']]

player_stats_2022_college_clean.to_csv(f'{path}player_stats_2022_college_clean.csv', encoding='utf-8-sig')



In [8]:
player_stats_2022_country_data = player_stats_2022.copy()

player_stats_2022_country_data['Country'] = player_stats_2022_country_data['Country'].str.replace('USA','US')
player_stats_2022_country_data['Country'] = player_stats_2022_country_data['Country'].str.replace('Argentina','AR')
player_stats_2022_country_data['Country'] = player_stats_2022_country_data['Country'].str.replace('United Kingdom','UK')

player_stats_2022_country_filter = player_stats_2022_country_data.loc[(player_stats_2022_country_data['Country'] != '') & (player_stats_2022_country_data['PER'] > 0)].reset_index(drop=True)

player_stats_2022_country_clean= player_stats_2022_college_filter[['Player','Country','PER']]

player_stats_2022_country_clean.to_csv(f'{path}player_stats_2022_country_clean.csv', encoding='utf-8-sig')

In [9]:
player_stats_2022_uscity_data = player_stats_2022.copy()

player_stats_2022_uscity_filter = player_stats_2022_uscity_data.loc[(player_stats_2022_uscity_data['Country'] == 'US') &
                                                                    (player_stats_2022_uscity_data['PER'] > 0) &
                                                                    (player_stats_2022_uscity_data['Player City'] != '') &
                                                                    (player_stats_2022_uscity_data['Team City'] != '') &
                                                                    (player_stats_2022_uscity_data['State'] != '') ].reset_index(drop=True)

player_stats_2022_uscity_clean= player_stats_2022_uscity_filter[['Player','Player City','Country','State','Team City','PER']]

player_stats_2022_uscity_clean['Player US Full City'] = player_stats_2022_uscity_clean.apply(lambda x:'%s, %s' % (x['Player City'],x['State']),axis=1)

player_stats_2022_uscity_clean.to_csv(f'{path}player_stats_2022_uscity_clean.csv', encoding='utf-8-sig')

C:\Users\dmostacero\Anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()


In [10]:
player_stats_2022_intcity_data = player_stats_2022.copy()

player_stats_2022_intcity_filter = player_stats_2022_intcity_data.loc[(player_stats_2022_uscity_data['Country'] != 'US') &
                                                                    (player_stats_2022_uscity_data['Country'] != '') &
                                                                    (player_stats_2022_uscity_data['PER'] > 0) &
                                                                    (player_stats_2022_uscity_data['Player City'] != '') &
                                                                    (player_stats_2022_uscity_data['Team City'] != '')].reset_index(drop=True)

player_stats_2022_intcity_clean= player_stats_2022_intcity_filter[['Player','Player City','Country','Team City','PER']]

player_stats_2022_intcity_clean['Player Int Full City'] = player_stats_2022_intcity_clean.apply(lambda x:'%s, %s' % (x['Player City'],x['Country']),axis=1)

player_stats_2022_intcity_clean.to_csv(f'{path}player_stats_2022_intcity_clean.csv', encoding='utf-8-sig')

C:\Users\dmostacero\Anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
